In [1]:
# Imports
from pathlib import Path
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

In [2]:
def relu(x):
    return np.maximum(0,x)

In [3]:
class TwoLayerNet():
    def __init__(self,input_size,hidden_size,output_size,wi=1e-4):
        self.W1 = wi * np.random.randn(input_size,hidden_size)
        self.W2 = wi * np.random.randn(hidden_size,output_size)
        self.b1 = np.zeros(hidden_size)
        self.b2 = np.zeros(output_size)
        
    def loss(self, X, y = None, reg=0.0):
        length, width = X.shape
        
        W1 = self.W1
        W2 = self.W2
        b1 = self.b1
        b2 = self.b2
        
        y1 = relu(X.dot(W1) + b1)
        y2 = y1.dot(W2) + b2
        
        if (y is None):
            return y2
        
        W1 = self.W1
        W2 = self.W2
        b1 = self.b1
        b2 = self.b2
        length, width = X.shape

        
        y1 = relu(X.dot(W1) + b1) 
        y2 = y1.dot(W2) + b2

         
        if(y is None): 
            return y2

        y2 -= y2.max()

        y2 = np.exp(y2)
        sumexp_y2 = np.sum(y2, axis=1)

        
        sm = y2 / sumexp_y2.reshape(length,1) 
        loss =  (-1.0) * np.sum(np.log(sm[range(length),y]))

         
        loss /= length 
        loss += reg * np.sum(W1 * W1) 
        loss += reg * np.sum(W2 * W2) 

        correct_class_scores = y2[range(length), y]
        sm[range(length), y] = (-1.0) * (sumexp_y2 - correct_class_scores)/sumexp_y2
        sm /= length


        grads_W2 = y1.T.dot(sm)
        grads_b2 = np.sum(sm, axis=0)
        grads_W2 += reg * 2 * W2

        hidden = sm.dot(W2.T)

        hidden[y1 == 0] = 0 

        grads_W1 = X.T.dot(hidden) 
        grads_b1 = np.sum(hidden, axis=0) 
        grads_W1 += reg * 2 * W1

        return loss, grads_W1, grads_b1, grads_W2, grads_b2


    def train(self, X, y, X_val, y_val, lr=1e-3, lr_decay=0.95, 
             reg=5e-6, num_iters = 100, batch_size=200,
             it_verbose = 1, verbose = False):
        
        length, width = X.shape
        val_length = X_val.shape[0]
        iteration_per_epoch = max(length/batch_size, 1)
        
        loss_hist = []
        train_acc_hist = []
        val_acc_hist = []
        
        for it in range(num_iters):
            s = np.random.choice(np.arange(length), batch_size, replace=False)
            X_batch = X[s]
            y_batch = y[s]
        
            loss, grads_W1, grads_b1, grads_W2, grads_b2 = self.loss(X_batch, y=y_batch, reg=reg)
            loss_hist.append(loss)
            
            self.W1 += (-1.0) * lr * grads_W1  
            self.b1 += (-1.0) * lr * grads_b1
            self.W2 += (-1.0) * lr * grads_W2  
            self.b2 += (-1.0) * lr * grads_b2
            
            if (verbose and it&it_verbose==0):
                print('iteration: %d / %d | Loss: %f' % (it, num_iters, loss))
            
            if (it%iteration_per_epoch==0):
                train_acc = (self.predict(X_batch) == y_batch).mean()
                val_acc = (self.predict(X_val) == y_val).mean()
                train_acc_hist.append(train_acc)
                val_acc_hist.append(val_acc)
                
                lr *= lr_decay
                
        return loss_hist, train_acc_hist, val_acc_hist
    
    def predict(self, X):
        
        y_pred = None
        
        W1 = self.W1
        W2 = self.W2
        b1 = self.b1
        b2 = self.b2

        y1 = relu(X.dot(W1) + b1)
        y2 = y1.dot(W2) + b2
        y_pred = np.argmax(y2, axis = 1)

        return y_pred

In [4]:
data_dir = './dataset'
p = Path(data_dir)

data_paths = [x for x in p.iterdir() if x.is_dir()]

classes = []
for i in range(len(data_paths)):
    classes.append(data_paths[i].name)
num_data = 300

count = 0
for i in range(len(data_paths)):
    count += len([x for x in data_paths[0].iterdir() if x.glob('./*.tif')])

y = np.zeros(num_data * len(data_paths))

for c in range(len(data_paths)):
    start = c * num_data
    end = start + num_data
    for i in range(start, end):
        y[i] = c

In [5]:
# Image dimension
im_width = 200
im_height = 200 
im_channels = 3
im_dim = im_width * im_height * im_channels

In [6]:
X = np.ndarray(shape=(len(data_paths) * num_data, im_width, im_height, im_channels), dtype=np.float64)

for c in range(len(data_paths)):
    start = c * num_data
    for i in range(num_data):
        f = str(list(data_paths[c].glob('**/*.tif'))[i])
        X[start + i] = cv2.resize(cv2.imread(f), (im_width, im_height))
    
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)

del X
del y

In [7]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

X_train = np.reshape(X_train, (X_train.shape[0], -1)).astype('float64')
X_test = np.reshape(X_test, (X_test.shape[0], -1)).astype('float64')

In [8]:
# Train the model
model = TwoLayerNet(im_dim, 200, 4)
loss_hist, train_acc_hist, val_acc_hist = model.train(X_train, y_train, X_test, y_test)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in log


In [9]:
# Prepare images for prediction
X_pred = np.ndarray(shape=(2, im_width, im_height, im_channels), dtype=np.float64)
X_label = ['', '']
for i in range(0,2):
    f = str(list(data_paths[i+2].iterdir())[-5])
    X_pred[i] = cv2.resize(cv2.imread(f), (im_width, im_height))
    X_label[i] = classes[i]


In [10]:
pred = model.predict(np.reshape(X_pred, (X_pred.shape[0], -1)).astype('float64'))

In [11]:
for i in range(0,2):
    print(classes[pred[i]])
    

soybean
soybean
